## 一、数据集切分
### 交叉验证
10W训练集分成8W和2W,8W用于第一层模型交叉验证，2W用于第二层模型交叉验证

## 二、算法流程
### 预处理
1. 用LR填充空缺目标值，合并train test数据集
2. 训练dbow和dm ２种doc2vec特征

### 训练第一层模型
1. 训练tfidf-lr模型
2. 训练dbow-nn模型
3. 训练dm-nn模型

### 训练第二层模型
1. 训练 xgb-ens模型

In [2]:
'''1.concat train data and test data 
   2.use lr to fill null label'''

import pandas as pd
import numpy as np
import jieba
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedShuffleSplit,StratifiedKFold,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import pickle
import cfg

#----------------------load data--------------------------------
df_tr = []
for i,line in enumerate(open(cfg.data_path + 'user_tag_query.10W.TRAIN',encoding='GB18030')):
    segs = line.split('\t')
    row = {}
    row['Id'] = segs[0]
    row['age'] = int(segs[1])
    row['gender'] = int(segs[2])
    row['Education'] = int(segs[3])
    row['query'] = '\t'.join(segs[4:])
    df_tr.append(row)
df_tr = pd.DataFrame(df_tr)

df_te=[]
for i,line in enumerate(open(cfg.data_path + 'user_tag_query.10W.TEST',encoding='GB18030')):
    segs = line.split('\t')
    row = {}
    row['Id'] = segs[0]
    row['query'] = '\t'.join(segs[1:])
    df_te.append(row)
df_te = pd.DataFrame(df_te)

print(df_tr.shape)
print(df_te.shape)

df_all = df_tr
# df_all = pd.concat([df_tr,df_te]).fillna(1)
# df_all.index = range(len(df_all))

for lb in ['Education','age','gender']:
    df_all[lb] = df_all[lb] - 1
    print(df_all.iloc[:100000][lb].value_counts())
    
class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1,2]:
                for i in range(len(words) - gram + 1):
                    tokens += ["_*_".join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%10000==0:
            print(self.n,end=' ')
        return tokens    

tfv = TfidfVectorizer(tokenizer=Tokenizer(),min_df=3,max_df=0.95,sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])
# pickle.dump(X_sp,open(root + 'tfidf_10W.pkl','wb'))
print(len(tfv.vocabulary_))
X_all = X_sp

#-----------------------------fill nan-------------------------------------
'''填充空值'''
for lb,idx in [('Education',0),('age',2),('gender',3)]:
    tr = np.where(df_all[lb]!=-1)[0]
    va = np.where(df_all[lb]==-1)[0]
lb = 'Education'
idx = 0
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=1).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

lb = 'age'
idx = 2
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=2).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

lb = 'gender'
idx = 3
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=2).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

df_all = pd.concat([df_all,df_te]).fillna(0)
df_all.to_csv(cfg.data_path + 'all_v2.csv',index=None,encoding='utf8')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lhc\AppData\Local\Temp\jieba.cache


(100000, 5)
(100000, 2)
 4    37107
 3    28148
 2    18858
-1     9280
 5     5693
 1      560
 0      354
Name: Education, dtype: int64
 0    38996
 1    26744
 2    18529
 3    10654
 4     2922
-1     1666
 5      489
Name: age, dtype: int64
 0    56976
 1    40869
-1     2155
Name: gender, dtype: int64


Loading model cost 0.735 seconds.
Prefix dict has been built succesfully.


10000 20000 30000 40000 50000 60000 丰田中东和中规	江西五十铃	奇瑞	江淮	帕杰罗劲畅	江玲好还是五十铃好	打大树和打沙袋的区别	一个月赚多少钱能开起霸道	商品房合同上面写享受房交会价钱	天籁和雅阁哪个好	驭胜乞丐版	微信摇一摇骗子	2016款途观	江铃驭胜怎么样	创步沙袋	租车有什么说道吗	底盘很高的越野车	福特猛禽	鞍山哪检三轮车	炖鱼为什么会碎	福特撼路者	星月圆珠子都是水磨吗	斯里兰卡好玩吗 沙滩	莲子心泡水喝的功效	全款买房延期怎么算	十万元左右的suv车	酷路泽4000中东版价格	江西五十铃mu x视频	驭胜s3502016款柴油	电脑无限封逆战号	丰田	呐志捷大7	鞍山租车凯美瑞	大显suv	驭胜有四驱的吗	架子鼓女神雅研	十万左右的二手suv	广汽传琪	驭胜新款好还是老款好	驭胜	中华十一绝牛人都是谁	迈腾	mg	驭胜好吗	奇骏2.5都是一样的发动机吗	鞍山铁西教散打的	鞍山靓号网	大石桥车能在鞍山检车吗	江西五十铃mx	本田crv	鞍山江西五十铃	立式沙袋一般多高	鱼怎么炖好吃	很大的吉普不是很贵	中牧水龙头	鞍山租车	一汽丰田和广汽丰田哪个好	江淮大好运4.2米报价和车图	五十铃	鞍山五十铃4s店地址	北京40视频在线观看	索兰托	中华十一绝老大是谁	北京b40	二手霸道2700	本田几代好	雅阁方向机有问题吗	江玲	沙袋太矮怎么打	2016suv销量排行榜	煤油打火机调火石	大力金刚腿怎么练	江西五十铃mu-x好不好	呐志杰u7	二手霸道值得入手吗	s350驭胜	奇骏没防撞梁	中华十一绝牛人排行	本田雅阁几代最好	瑞麒	2700霸道废油吗	18万买什么suv车好	中华十一绝都有谁	乞丐版途观好不好	铁砂掌怎么练	jeep	本田crv好不好	莲子鱼的做法大全	中华十一绝人物介绍	2,	十万买什么轿车最经济	鞍山铁西教拳击的	连子鱼怎么做好吃	奇骏大事故到底抗撞吗	最保值的suv	凯美瑞好还是雅阁好	斯巴鲁	驭胜s350缺点	驭胜和江西五十铃	手动新途观	苹果四声音太小怎么办	斯里兰卡是哪个国家	暂停服务是什么意思	大众途观1.8手动	奥迪a4	雅阁九代有问题吗	五菱荣光换怠速马达多少钱	可以越野中型suv车型	五菱荣光换胎速马达多少钱	鞍山教自由搏击的	呐志捷	江西五十铃mu-x	学卖二手车好学吗	中牧和伟星	

In [16]:
'''train dbow/dm for education/age/gender'''

import pandas as pd
import jieba
from datetime import datetime
from collections import namedtuple
from gensim.models import Doc2Vec
from collections import OrderedDict
import subprocess
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
import codecs
import cfg
import numpy as np

df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8')
#-------------------add row number to query----------------------
doc_f = codecs.open('alldata-id.txt','w',encoding='utf8')
for i,queries in enumerate(df_all.iloc[:100000]['query']):
    words = []
    for query in queries.split('\t'):
        words.extend(list(jieba.cut(query)))
    tags = [i]
    if i % 10000 == 0:
        print(datetime.now(),i)
    doc_f.write('_*{} {}'.format(i,' '.join(words)))
doc_f.close()

#-------------------------prepare to train--------------------------------------------
def run_cmd(cmd):
    print(cmd)
    process = subprocess.Popen(cmd, shell=True,
                       stdout=subprocess.PIPE,
                        stderr=subprocess.STDOUT)
    for t, line in enumerate(iter(process.stdout.readline,b'')):
        line = line.decode('utf8').rstrip()
        print(line)
    process.communicate()
    return process.returncode

SentimentDocument = namedtuple('SentimentDocument', 'words tags')
class Doc_list(object):
    def __init__(self,f):
        self.f = f
    def __iter__(self):
        for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
            words = line.split()
            tags = [int(words[0][2:])]
            words = words[1:]
            yield SentimentDocument(words,tags)
d2v = Doc2Vec(dm=0, size=300, negative=5, hs=0, min_count=3, window=30,sample=1e-5,workers=8,alpha=0.025,min_alpha=0.025)
doc_list = Doc_list('alldata-id.txt')
d2v.build_vocab(doc_list)

#-------------------train dbow doc2vec---------------------------------------------
df_lb = pd.read_csv(cfg.data_path + 'all_v2.csv',usecols=['Education','age','gender'],nrows=100000)
ys = {}
for lb in ['Education','age','gender']:
    ys[lb] = np.array(df_lb[lb])

for i in range(2):
    print(datetime.now(),'pass:',i + 1)
#     run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
    doc_list = Doc_list('alldata-id.txt')
    d2v.train(doc_list)
    X_d2v = np.array([d2v.docvecs[i] for i in range(100000)])
    for lb in ["Education",'age','gender']:
        scores = cross_val_score(LogisticRegression(C=3),X_d2v,ys[lb],cv=5)
        print('dbow',lb,scores,np.mean(scores))
d2v.save(cfg.data_path + 'dbow_d2v.model')
print(datetime.now(),'save done')

d2v = Doc2Vec(dm=1, size=300, negative=5, hs=0, min_count=3, window=10,sample=1e-5,workers=8,alpha=0.05,min_alpha=0.025)
doc_list = Doc_list('alldata-id.txt')
d2v.build_vocab(doc_list)

#---------------train dm doc2vec-----------------------------------------------------
for i in range(10):
    print(datetime.now(),'pass:',i)
#     run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
    doc_list = Doc_list('alldata-id.txt')
    d2v.train(doc_list)
    X_d2v = np.array([d2v.docvecs[i] for i in range(100000)])
    for lb in ["Education",'age','gender']:
        scores = cross_val_score(LogisticRegression(C=3),X_d2v,ys[lb],cv=5)
        print('dm',lb,scores,np.mean(scores))
d2v.save(cfg.data_path + 'dm_d2v.model')
print(datetime.now(),'save done')

C:\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lhc\AppData\Local\Temp\jieba.cache
Loading model cost 0.700 seconds.
Prefix dict has been built succesfully.


2017-01-12 15:36:29.419356 0
2017-01-12 15:37:24.815547 10000
2017-01-12 15:38:20.066305 20000
2017-01-12 15:39:12.412226 30000
2017-01-12 15:40:08.696306 40000
2017-01-12 15:41:04.784937 50000
2017-01-12 15:42:02.058357 60000
2017-01-12 15:42:56.711774 70000
2017-01-12 15:43:49.831915 80000
2017-01-12 15:44:40.984926 90000
2017-01-12 15:46:00.153113 pass: 1
dbow Education [ 0.62836858  0.62716864  0.62511874  0.62743137  0.63236324] 0.628090115019
dbow age [ 0.58209179  0.57754225  0.58277086  0.58365837  0.58643797] 0.582500245917
dbow gender [ 0.8343   0.8301   0.8357   0.83895  0.8347 ] 0.83475
2017-01-12 16:01:27.135225 pass: 2
dbow Education [ 0.63476826  0.6359682   0.6339683   0.63318166  0.63336334] 0.634249952036
dbow age [ 0.59129087  0.58759124  0.58482076  0.58730873  0.59193879] 0.588590078489
dbow gender [ 0.8367   0.8344   0.83825  0.8417   0.83895] 0.838
2017-01-12 16:11:33.578202 save done
2017-01-12 16:12:00.556195 pass: 0
dm Education [ 0.59062047  0.59892005  0.595

In [8]:
'''tfidf-lr stack for education/age/gender'''

import pandas as pd
import numpy as np
import jieba
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import KFold
from datetime import datetime
import cfg

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)
#-----------------------load data--------------------

df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8',nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])

class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1,2]:
                for i in range(len(words) - gram + 1):
                    tokens += ["_*_".join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%10000==0:
            print(self.n)
        return tokens

tfv = TfidfVectorizer(tokenizer=Tokenizer(),min_df=3,max_df=0.95,sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])
pickle.dump(X_sp,open(cfg.data_path + 'tfidf_10W.feat','wb'))

df_stack = pd.DataFrame(index=range(len(df_all)))

#-----------------------stack for education/age/gender------------------
for lb in ['Education','age','gender']:
    print(lb)
    TR = 80000
    num_class = len(pd.value_counts(ys[lb]))
    n = 5

    X = X_sp[:TR]
    y = ys[lb][:TR]
    X_te = X_sp[TR:]
    y_te = ys[lb][TR:]

    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))

    for i,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('%s stack:%d/%d'%(str(datetime.now()),i+1,n))
        clf = LogisticRegression(C=3)
        clf.fit(X[tr],y[tr])
        y_pred_va = clf.predict_proba(X[va])
        y_pred_te = clf.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['tfidf_{}_{}'.format(lb,i)] = stack_all[:,i]

df_stack.to_csv(cfg.data_path + 'tfidf_stack_10W.csv',index=None,encoding='utf8')
print(datetime.now(),'save tfidf stack done!')

10000
20000
30000
40000
50000
60000
不灭剑祖	六间房	南艺新生颜值爆表	阴模	河南现纸片楼	苹果手机定位追踪	昔日港姐曝黑幕	大主宰	武神空间	库克抛售苹果股份	女演员徐婷病逝	熊孩甜馨自己理发	王宝强走出阴霾	剑道通神txt下载	张翰家门长蘑菇	普京专车发生事故	法师真理	永恒剑主	宁波二	万古仙穹	三蛮里的小六子	透支9元还9000	为女儿造血生3子	朝鲜疑发生核爆	从容淡定的某人	雄霸蛮荒	阿根廷捕获奇特鱼	玄界之门	雪鹰领主	男子扛妻遗体步行	不朽凡人	六间房秀场直播大厅	美又收天价核潜艇	滨崎步宣布离婚	商物模特	凡人寻道	陈乔恩遭网友炮轰	钓鱼视频	万域之王	日本进入多死社会	世界第一高桥合龙	三毛里的小六子	菲总统向中国求情	天气预报	台风把飞机吹跑	老虎咬人调查结果	武破万古	武林半侠传	买婚房却买到凶宅	李多海承认恋情	刘翔被曝求婚成功	人道至尊	渔民捕获怪头鱼	出租绕道罚2000	马蓉秒删微博	刘子歌将嫁恩师	福建土豪办婚礼	女生求婚大爷	飞豹战机赴俄参赛	造化之王	瑞士战机失踪	麻雀收视火爆	马云报仇买肯德基	观众体验残奥项目	剑道通神笔趣阁	神藏	剑道通神	王宝强财产分割	神马电影	武炼巅峰	香港慰问驻港部队	一念永恒	永恒之心	男子术后双肾全无	面部移植者辞世	曼联计划贱卖鲁尼	青岛对虾论只卖	超凡传	世界最美杀马特	美国纽约发生爆炸	巴基斯坦捐赠大米	商务模特	女司机草根遮号牌	女孩放生六千头羊	普京送安倍武士刀	真武世界	神	朝鲜代表归国遇冷	眼镜王蛇占屋为王	连撞三车称遇到鬼	凡人修仙	傅园慧收贝尔战书	宋小宝耍大牌被殴	德国难民街头斗殴	古力娜扎关闭评论	巫界之树	黄渤携妻女会友	厦门重启限购	剑游太虚	美印签署后勤协定	永恒圣帝	纪晓岚和珅重聚	剑主苍穹	大话3遭吐槽	炸鸡加罂粟壳判刑	柯震东李毓芬复合	龙符

['不灭', '剑祖', '不灭_*_剑祖', '六间房', '南艺', '新生', '颜值', '爆表', '南艺_*_新生', '新生_*_颜值', '颜值_*_爆表', '阴模', '河南', '现', '纸片', '楼', '河南_*_现', '现_*_纸片', '纸片_*_楼', '苹果', '手机', '定位', '追踪', '苹果_*_手机', '手机_*_定位', '定位

In [1]:
'''dbow-nn stack for education/age/gender'''

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import KFold
from gensim.models import Doc2Vec
from collections import OrderedDict

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import re
import cfg

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)

#-----------------------load dataset----------------------
df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])
    
model = Doc2Vec.load(cfg.data_path + 'dbow_d2v.model')
X_sp = np.array([model.docvecs[i] for i in range(100000)])

#----------------------dbowd2v stack for Education/age/gender---------------------------
df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 80000
n = 5

X = X_sp[:TR]
X_te = X_sp[TR:]

feat = 'dbowd2v'
for i,lb in enumerate(['Education','age','gender']):
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]
    
    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))
    
    for k,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('{} stack:{}/{}'.format(datetime.now(),k+1,n))
        nb_classes = num_class
        X_train = X[tr]
        y_train = y[tr]
        X_test = X_te
        y_test = y_te

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)


        model = Sequential()
        model.add(Dense(300,input_shape=(X_train.shape[1],)))
        model.add(Dropout(0.1))
        model.add(Activation('tanh'))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adadelta',
                      metrics=['accuracy'])

        history = model.fit(X_train, Y_train,shuffle=True,
                            batch_size=128, nb_epoch=35,
                            verbose=2, validation_data=(X_test, Y_test))
        y_pred_va = model.predict_proba(X[va])
        y_pred_te = model.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for l in range(stack_all.shape[1]):
        df_stack['{}_{}_{}'.format(feat,lb,l)] = stack_all[:,l]
df_stack.to_csv(cfg.data_path + 'dbowd2v_stack_10W.csv',encoding='utf8',index=None)
print(datetime.now(),'save dbowd2v stack done!')

C:\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.


2017-01-12 20:25:22.517795 stack:1/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
2s - loss: 0.9891 - acc: 0.6110 - val_loss: 0.9312 - val_acc: 0.6365
Epoch 2/35
2s - loss: 0.9429 - acc: 0.6302 - val_loss: 0.9321 - val_acc: 0.6370
Epoch 3/35
2s - loss: 0.9359 - acc: 0.6340 - val_loss: 0.9285 - val_acc: 0.6374
Epoch 4/35
2s - loss: 0.9327 - acc: 0.6346 - val_loss: 0.9272 - val_acc: 0.6372
Epoch 5/35
2s - loss: 0.9298 - acc: 0.6359 - val_loss: 0.9246 - val_acc: 0.6384
Epoch 6/35
2s - loss: 0.9249 - acc: 0.6383 - val_loss: 0.9243 - val_acc: 0.6413
Epoch 7/35
2s - loss: 0.9224 - acc: 0.6387 - val_loss: 0.9264 - val_acc: 0.6401
Epoch 8/35
2s - loss: 0.9195 - acc: 0.6403 - val_loss: 0.9212 - val_acc: 0.6415
Epoch 9/35
2s - loss: 0.9155 - acc: 0.6419 - val_loss: 0.9187 - val_acc: 0.6427
Epoch 10/35
2s - loss: 0.9121 - acc: 0.6416 - val_loss: 0.9164 - val_acc: 0.6443
Epoch 11/35
2s - loss: 0.9084 - acc: 0.6442 - val_loss: 0.9163 - val_acc: 0.6425
Epoch 12/35
2s - loss: 0.9060 -

In [2]:
'''dm-nn stack for education/age/gender'''

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import KFold
from gensim.models import Doc2Vec
from collections import OrderedDict

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import re
import cfg

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)

#-----------------------load dataset----------------------
df_all = pd.read_csv(cfg.data_path + 'all.csv',encoding='utf8',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])
    
model = Doc2Vec.load(cfg.data_path + 'dm_d2v.model')
X_sp = np.array([model.docvecs[i] for i in range(100000)])

#----------------------dmd2v stack for Education/age/gender---------------------------

df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 80000
n = 5

X = X_sp[:TR]
X_te = X_sp[TR:]

feat = 'dmd2v'
for i,lb in enumerate(['Education','age','gender']):
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]
    
    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))
    
    for k,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('{} stack:{}/{}'.format(datetime.now(),k+1,n))
        nb_classes = num_class
        X_train = X[tr]
        y_train = y[tr]
        X_test = X_te
        y_test = y_te

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)


        model = Sequential()
        model.add(Dense(300,input_shape=(X_train.shape[1],)))
        model.add(Dropout(0.1))
        model.add(Activation('tanh'))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adadelta',
                      metrics=['accuracy'])

        history = model.fit(X_train, Y_train,shuffle=True,
                            batch_size=128, nb_epoch=35,
                            verbose=2, validation_data=(X_test, Y_test))
        y_pred_va = model.predict_proba(X[va])
        y_pred_te = model.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for l in range(stack_all.shape[1]):
        df_stack['{}_{}_{}'.format(feat,lb,l)] = stack_all[:,l]
df_stack.to_csv(cfg.data_path + 'dmd2v_stack_10W.csv',encoding='utf8',index=None)
print(datetime.now(),'save dmd2v stack done!')

2017-01-12 20:51:05.276891 stack:1/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
2s - loss: 1.0036 - acc: 0.6026 - val_loss: 0.9368 - val_acc: 0.6299
Epoch 2/35
2s - loss: 0.9425 - acc: 0.6299 - val_loss: 0.9282 - val_acc: 0.6383
Epoch 3/35
2s - loss: 0.9373 - acc: 0.6314 - val_loss: 0.9264 - val_acc: 0.6380
Epoch 4/35
2s - loss: 0.9354 - acc: 0.6316 - val_loss: 0.9254 - val_acc: 0.6412
Epoch 5/35
2s - loss: 0.9331 - acc: 0.6324 - val_loss: 0.9316 - val_acc: 0.6332
Epoch 6/35
2s - loss: 0.9315 - acc: 0.6331 - val_loss: 0.9288 - val_acc: 0.6385
Epoch 7/35
2s - loss: 0.9301 - acc: 0.6337 - val_loss: 0.9243 - val_acc: 0.6410
Epoch 8/35
2s - loss: 0.9285 - acc: 0.6350 - val_loss: 0.9275 - val_acc: 0.6394
Epoch 9/35
2s - loss: 0.9267 - acc: 0.6338 - val_loss: 0.9280 - val_acc: 0.6375
Epoch 10/35
2s - loss: 0.9261 - acc: 0.6345 - val_loss: 0.9236 - val_acc: 0.6421
Epoch 11/35
2s - loss: 0.9245 - acc: 0.6356 - val_loss: 0.9250 - val_acc: 0.6391
Epoch 12/35
2s - loss: 0.9243 -

In [3]:
'''xgb-ens for education/age/gender'''

import pandas as pd
import numpy as np
import xgboost as xgb
import cfg
import datetime

def xgb_acc_score(preds,dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds,axis=1)
    return [('acc',np.mean(y_true == y_pred))]

df_lr = pd.read_csv(cfg.data_path + 'tfidf_stack_10W.csv')
df_dm = pd.read_csv(cfg.data_path + 'dmd2v_stack_10W.csv')
df_dbow = pd.read_csv(cfg.data_path + 'dbowd2v_stack_10W.csv')

df_lb = pd.read_csv(cfg.data_path + 'all.csv',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for lb in ['Education','age','gender']:
    ys[lb] = np.array(df_lb[lb])

'''最好的参数组合'''
#-------------------------education----------------------------------
TR = 80000
df_sub = pd.DataFrame()
df_sub['Id'] = df_lb.iloc[TR:]['Id']
seed = 10
lb = 'Education'
print(lb)

esr = 25
evals = 1
n_trees = 1000

df = pd.concat([df_lr,df_dbow,df_dm],axis=1)
print(df.columns)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.9
mc = 2
md = 8
gm = 2
# n_trees = 30

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':0.8,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['Education'] = np.argmax(bst.predict(dvalid),axis=1) + 1
#------------------------ age-----------------------------------
lb = 'age'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

# df = pd.concat([df_stack_tfidf,df_stack_d2v],axis=1)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.5
mc = 3
md = 7
gm = 2
# n_trees = 37

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':1,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['age'] = np.argmax(bst.predict(dvalid),axis=1)+1
#--------------------------gender-------------------------------------
lb = 'gender'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

# df = pd.concat([df_lr,df_multid2v],axis=1)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]


ss = 0.5
mc = 0.8
md = 7
gm = 1
# n_trees = 25

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':1,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['gender'] = np.argmax(bst.predict(dvalid),axis=1)+1

df_sub = df_sub[['Id','age','gender','Education']]
df_sub.to_csv(cfg.data_path + 'tfidf_dm_dbow_2W.csv',index=None,header=None,sep=' ')

Education
Index(['tfidf_Education_0', 'tfidf_Education_1', 'tfidf_Education_2',
       'tfidf_Education_3', 'tfidf_Education_4', 'tfidf_Education_5',
       'tfidf_age_0', 'tfidf_age_1', 'tfidf_age_2', 'tfidf_age_3',
       'tfidf_age_4', 'tfidf_age_5', 'tfidf_gender_0', 'tfidf_gender_1',
       'dbowd2v_Education_0', 'dbowd2v_Education_1', 'dbowd2v_Education_2',
       'dbowd2v_Education_3', 'dbowd2v_Education_4', 'dbowd2v_Education_5',
       'dbowd2v_age_0', 'dbowd2v_age_1', 'dbowd2v_age_2', 'dbowd2v_age_3',
       'dbowd2v_age_4', 'dbowd2v_age_5', 'dbowd2v_gender_0',
       'dbowd2v_gender_1', 'dmd2v_Education_0', 'dmd2v_Education_1',
       'dmd2v_Education_2', 'dmd2v_Education_3', 'dmd2v_Education_4',
       'dmd2v_Education_5', 'dmd2v_age_0', 'dmd2v_age_1', 'dmd2v_age_2',
       'dmd2v_age_3', 'dmd2v_age_4', 'dmd2v_age_5', 'dmd2v_gender_0',
       'dmd2v_gender_1'],
      dtype='object')
[0]	train-merror:0.315875	eval-merror:0.34185	train-acc:0.684125	eval-acc:0.65815
Multiple e